In [1]:
import os
import yaml
import json
from dataclasses import asdict
import numpy as np
from spflows import results_path
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import MLFlowLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from spflows.utils.experiment_files import ExperimentsFiles
from spflows.configs_classes.forecasting_configs import ForecastingModelConfig
from spflows.data.datamodules import ForecastingDataModule
from spflows.training.basic_experiments import BasicLightningExperiment

from spflows.models.forecasting.score_lightning import ScoreModule
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

c:\Users\cesar\anaconda3\envs\torchts\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
experiment_dir=r"C:\Users\cesar\Desktop\Projects\BirthDeathPortfolio\stochastic_process_flows\results\1734007921"
experiment = BasicLightningExperiment(experiment_dir=experiment_dir)

In [3]:
databatch = experiment.datamodule.get_train_databatch()
device = next(experiment.model.parameters().__iter__()).device
inputs = [v.to(device) for v in databatch.values()]
loss = experiment.model.train_dynamical_module(*inputs)

In [4]:
experiment.evaluate()

Running evaluation: 7it [00:00, 203.33it/s]
Running evaluation: 7it [00:00, 220.91it/s]
Running evaluation: 7it [00:00, 215.37it/s]
Running evaluation: 7it [00:00, 201.04it/s]
Running evaluation: 7it [00:00, 227.29it/s]
Running evaluation: 7it [00:00, 208.70it/s]
Running evaluation: 7it [00:00, 211.72it/s]
Running evaluation: 7it [00:00, 221.94it/s]
Running evaluation: 7it [00:00, 221.35it/s]
Running evaluation: 7it [00:00, 222.78it/s]
Running evaluation: 7it [00:00, 224.65it/s]
Running evaluation: 7it [00:00, 220.72it/s]
Running evaluation: 7it [00:00, 240.86it/s]
Running evaluation: 7it [00:00, 222.06it/s]
Running evaluation: 7it [00:00, 221.95it/s]
Running evaluation: 7it [00:00, 215.87it/s]
Running evaluation: 7it [00:00, 215.22it/s]
Running evaluation: 7it [00:00, 233.00it/s]
Running evaluation: 7it [00:00, 239.59it/s]
Running evaluation: 7it [00:00, 215.14it/s]
Running evaluation: 7it [00:00, 236.46it/s]
Running evaluation: 7it [00:00, 222.00it/s]
Running evaluation: 7it [00:00, 

{'CRPS': 0.2800007560261729,
 'ND': 0.39566404570699765,
 'NRMSE': 3.7716464036068,
 'CRPS_sum': 0.26320333955094716,
 'ND_sum': 0.35032736496902495,
 'NRMSE_sum': 0.3713814883597412,
 'energy_score': 36478.62890625}

In [5]:
def energy_score(forecast, target):
    obs_dist = np.mean(np.linalg.norm((forecast - target), axis=-1))
    pair_dist = np.mean(
        np.linalg.norm(forecast[:, np.newaxis, ...] - forecast, axis=-1)
    )
    return obs_dist - pair_dist * 0.5

score = energy_score(
    forecast=np.array([x.samples for x in forecasts]),
    target=np.array([x[-experiment.datamodule.prediction_length:] for x in targets])[:,None,...],
)

evaluator = MultivariateEvaluator(quantiles=(np.arange(20)/20.0)[1:], target_agg_funcs={'sum': np.sum})
agg_metric, _ = evaluator(targets, forecasts, num_series=len(experiment.datamodule.test_data))

metrics = dict(
    CRPS=agg_metric['mean_wQuantileLoss'],
    ND=agg_metric['ND'],
    NRMSE=agg_metric['NRMSE'],
    CRPS_sum=agg_metric['m_sum_mean_wQuantileLoss'],
    ND_sum=agg_metric['m_sum_ND'],
    NRMSE_sum=agg_metric['m_sum_NRMSE'],
    energy_score=score,
)
metrics = { k: float(v) for k,v in metrics.items() }

NameError: name 'forecasts' is not defined

In [ ]:
metrics

{'CRPS': 0.283687026354285,
 'ND': 0.40029560990965696,
 'NRMSE': 3.9189627371634685,
 'CRPS_sum': 0.2686682903834107,
 'ND_sum': 0.3525848243385241,
 'NRMSE_sum': 0.3772822513421044,
 'energy_score': 38427.546875}

In [ ]:
json.dump(metrics,
          open(experiment.experiment_files.metrics_path.format("gluonts_evaluator"),"w"))